In [28]:
import pandas as pd

In [29]:
PATH = r"data\transactions_data.xlsx"
sheet1 = pd.read_excel(PATH, sheet_name="transaction")
sheet2 = pd.read_excel(PATH, sheet_name="customer")
sheet1.head()

C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\4121451393.py:3: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  sheet2 = pd.read_excel(PATH, sheet_name="customer")


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


In [30]:
sheet2.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [31]:
PRODUCT_COLS = ['product_id', 'brand', 'product_line', 'product_class', \
                'product_size', 'standard_cost']

def set_surragate_id(table: pd.DataFrame, col_mame: str) -> pd.DataFrame:
    table.reset_index(inplace=True)
    table.drop('index', axis=1, inplace=True)
    table[col_mame] = table.index + 1
    return table

products = sheet1[PRODUCT_COLS]

products.drop_duplicates(inplace=True, ignore_index=True)
products = set_surragate_id(products, "product_surragate_id")
products

C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\1574875511.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products.drop_duplicates(inplace=True, ignore_index=True)
C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\1574875511.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.drop('index', axis=1, inplace=True)
C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\1574875511.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,product_id,brand,product_line,product_class,product_size,standard_cost,product_surragate_id
0,2,Solex,Standard,medium,medium,53.620000,1
1,3,Trek Bicycles,Standard,medium,large,388.920000,2
2,37,OHM Cycles,Standard,low,medium,248.820000,3
3,88,Norco Bicycles,Standard,medium,medium,381.100000,4
4,78,Giant Bicycles,Standard,medium,large,709.480000,5
...,...,...,...,...,...,...,...
199,70,Norco Bicycles,Road,medium,medium,206.350000,200
200,0,Solex,Standard,medium,medium,84.990000,201
201,0,Solex,Road,medium,medium,312.735016,202
202,0,Norco Bicycles,Standard,medium,medium,270.299988,203


In [32]:
products.standard_cost.describe()

count     203.000000
mean      552.436626
std       410.170161
min         7.210000
25%       215.030000
50%       464.720000
75%       778.690000
max      1759.850000
Name: standard_cost, dtype: float64

In [33]:
def get_surragate_id(item: pd.Series, foreign_table: pd.DataFrame, 
                     cols: list, surragate_id_col: str) -> int:
    try:
        ind = foreign_table[(foreign_table[cols] == item).all(axis=1)].index[0]
        id = foreign_table.loc[ind, surragate_id_col]
        return id
    except IndexError:
        return 0

sheet1['product_surragate_id'] = sheet1[PRODUCT_COLS].apply(lambda item: 
                                 get_surragate_id(item, products, PRODUCT_COLS, 
                                                  'product_surragate_id'), axis=1)

In [34]:
transactions = sheet1[['transaction_id', 'product_surragate_id', 'customer_id', \
                       'transaction_date', 'online_order', 'order_status', 'list_price']]

transactions.drop_duplicates(inplace=True, ignore_index=True)
# transactions.order_status.unique()
transactions

C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\2562788431.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions.drop_duplicates(inplace=True, ignore_index=True)


,transaction_id,product_surragate_id,customer_id,transaction_date,online_order,order_status,list_price
0,1,1,2950,2017-02-25,False,Approved,71.49
1,2,2,3120,2017-05-21,True,Approved,2091.47
2,3,3,402,2017-10-16,False,Approved,1793.43
3,4,4,3135,2017-08-31,False,Approved,1198.46
4,5,5,787,2017-10-01,True,Approved,1765.30
...,...,...,...,...,...,...,...
19995,19996,123,1018,2017-06-24,True,Approved,2005.66
19996,19997,99,127,2017-11-09,True,Approved,416.98
19997,19998,147,2284,2017-04-14,True,Approved,1636.90
19998,19999,42,2764,2017-07-03,False,Approved,227.88


In [35]:
ADDRESSES_COLS = ['address', 'postcode', 'state', 'country']

addresses = sheet2[['address', 'postcode', 'state', 'country']]

addresses.drop_duplicates(inplace=True, ignore_index=True)
addresses = set_surragate_id(addresses, "address_surragate_id")
addresses

C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\3952267249.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses.drop_duplicates(inplace=True, ignore_index=True)
C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\1574875511.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.drop('index', axis=1, inplace=True)
C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\1574875511.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,address,postcode,state,country,address_surragate_id
0,060 Morning Avenue,2016,New South Wales,Australia,1
1,6 Meadow Vale Court,2153,New South Wales,Australia,2
2,0 Holy Cross Court,4211,QLD,Australia,3
3,17979 Del Mar Point,2448,New South Wales,Australia,4
4,9 Oakridge Court,3216,VIC,Australia,5
...,...,...,...,...,...
3995,57042 Village Green Point,4511,QLD,Australia,3996
3996,87 Crescent Oaks Alley,2756,NSW,Australia,3997
3997,8194 Lien Street,4032,QLD,Australia,3998
3998,320 Acker Drive,2251,NSW,Australia,3999


In [36]:
sheet2['address_surragate_id'] = sheet2[ADDRESSES_COLS].apply(lambda item: 
                                 get_surragate_id(item, addresses, ADDRESSES_COLS, 
                                                  'address_surragate_id'), axis=1)

In [37]:
customers = sheet2[['customer_id', 'address_surragate_id', 'first_name', 'last_name', \
                    'gender', 'DOB', 'job_title', 'job_industry_category', 'wealth_segment', \
                    'deceased_indicator', 'owns_car', 'property_valuation']]

customers.drop_duplicates(inplace=True, ignore_index=True)
customers

C:\Users\Timur\AppData\Local\Temp\ipykernel_32528\2071388465.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers.drop_duplicates(inplace=True, ignore_index=True)


,customer_id,address_surragate_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,property_valuation
0,1,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,10
1,2,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,10
2,3,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,9
3,4,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,4
4,5,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,3996,Rosalia,Halgarth,Female,1975-08-09,VP Product Management,Health,Mass Customer,N,No,6
3996,3997,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,10
3997,3998,3998,Sarene,Woolley,U,NaT,Assistant Manager,IT,High Net Worth,N,No,7
3998,3999,3999,Patrizius,NaN,Male,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,7


In [43]:
table_names = ['transactions', 'products', 'customers', 'addresses']

for i, table in enumerate([transactions, products, customers, addresses]):
    table.to_csv(f"data\\{table_names[i]}.csv", index=False)